In [439]:
import pandas as pd
import plotly.express as px
import re
import datetime

from Project.Database import Db

In [440]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [441]:
consumption_condition = (lambda self: (
        (self["Units"] == "W") &
        self["Description"].str.contains("power consumption" or "used") &
        ~(self["Unnamed: 0"] == "Elec_PowerMicrowave") &
        ~(self["Unnamed: 0"] == "Elec_PowerRefrigerator") &
        ~(self["Unnamed: 0"] == "Elec_PowerClothesWasher") &
        ~self["Unnamed: 0"].str.contains("Elec_[\w]+HVAC") &
        (~self["Unnamed: 0"].str.contains("DHW_") |
         self["Unnamed: 0"].str.contains("DHW_[\w]+Total")) &
        (~self["Unnamed: 0"].str.contains("Load_") |
         ~self["Unnamed: 0"].str.contains("Load_Microwave") &
         self["Unnamed: 0"].str.contains("Load_[\w]+Standby"))))

consumption = meta.loc[consumption_condition]["Unnamed: 0"].tolist()

production = meta.loc[(meta['Parameter'] == "Power_Electrical") & (
    meta["Description"].str.contains("produced"))]["Unnamed: 0"].tolist()

In [442]:
year1_consumption_production = year1_hourly[["Timestamp"] + consumption + production].copy()

In [443]:
year1_consumption_production["Mean_consumption"] = year1_consumption_production[consumption].sum(1)
year1_consumption_production["Mean_production"] = year1_consumption_production[production].sum(1)

In [444]:
agg_consum_prod = year1_consumption_production[["Timestamp", "Mean_consumption", "Mean_production"]].copy()

In [445]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
agg_consum_prod['Timestamp'] = pd.to_datetime(agg_consum_prod['Timestamp'], errors="coerce", utc=True,
                                              format="%Y-%m-%d %H:%M:%S%z")

#Extracts hour of Timestamp.
agg_consum_prod["MonthOfTimestamp"] = agg_consum_prod.Timestamp.dt.month
agg_consum_prod["WeekdayOfTimestamp"] = agg_consum_prod.Timestamp.dt.dayofweek
agg_consum_prod["HourOfTimestamp"] = agg_consum_prod.Timestamp.dt.hour
agg_consum_prod

,Timestamp,Mean_consumption,Mean_production,MonthOfTimestamp,WeekdayOfTimestamp,HourOfTimestamp
0,2013-07-01 04:00:00+00:00,1006.317361,18.506595,7,0,4
1,2013-07-01 05:00:00+00:00,629.003143,18.494896,7,0,5
2,2013-07-01 06:00:00+00:00,976.929283,18.376221,7,0,6
3,2013-07-01 07:00:00+00:00,676.805590,18.470420,7,0,7
4,2013-07-01 08:00:00+00:00,707.015157,18.338783,7,0,8
...,...,...,...,...,...,...
8756,2014-07-01 00:00:00+00:00,1702.495519,42.879678,7,1,0
8757,2014-07-01 01:00:00+00:00,2516.681689,18.101134,7,1,1
8758,2014-07-01 02:00:00+00:00,1367.653306,18.409789,7,1,2
8759,2014-07-01 03:00:00+00:00,2501.732181,17.882693,7,1,3


In [446]:
agg_consum_prod["Proportion_of_consumption_production"] = agg_consum_prod["Mean_production"] - agg_consum_prod[
    "Mean_consumption"]
hourON = agg_consum_prod.groupby(["MonthOfTimestamp", "HourOfTimestamp"]).mean()
hourON.reset_index(inplace=True)

hourON

,MonthOfTimestamp,HourOfTimestamp,Mean_consumption,Mean_production,WeekdayOfTimestamp,Proportion_of_consumption_production
0,1,0,2985.868974,18.411803,3.000000,-2967.457171
1,1,1,2821.263247,18.540778,3.000000,-2802.722469
2,1,2,3585.170426,18.688463,3.000000,-3566.481963
3,1,3,3247.913173,18.735128,3.000000,-3229.178045
4,1,4,2600.602557,18.789207,3.000000,-2581.813350
...,...,...,...,...,...,...
283,12,19,1243.010613,1958.302925,2.935484,715.292311
284,12,20,1671.809133,1012.433992,2.935484,-659.375141
285,12,21,1102.800315,128.462439,2.935484,-974.337876
286,12,22,1126.469904,17.451051,2.935484,-1109.018853


In [447]:
import plotly.graph_objects as go

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=hourON["Proportion_of_consumption_production"],
    x=hourON["HourOfTimestamp"],
    y=hourON["MonthOfTimestamp"],
    colorscale="RdBu",
    zmid=0
)
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    yaxis_nticks=20
    # annotations=annotations,
)

fig.show()

In [448]:
agg_consum_prod["Proportion_of_consumption_production"] = agg_consum_prod["Mean_production"] - agg_consum_prod[
    "Mean_consumption"]
hourON = agg_consum_prod.groupby(["WeekdayOfTimestamp", "HourOfTimestamp"]).mean()
hourON.reset_index(inplace=True)

hourON

,WeekdayOfTimestamp,HourOfTimestamp,Mean_consumption,Mean_production,MonthOfTimestamp,Proportion_of_consumption_production
0,0,0,1617.205648,20.361184,6.576923,-1596.844464
1,0,1,1803.129874,18.003767,6.576923,-1785.126107
2,0,2,1822.823517,18.136637,6.576923,-1804.686880
3,0,3,1398.540173,18.205499,6.576923,-1380.334674
4,0,4,1284.909183,18.605451,6.584906,-1266.303732
...,...,...,...,...,...,...
163,6,19,2654.670086,4210.451504,6.576923,1555.781417
164,6,20,1774.606170,3066.155052,6.576923,1291.548882
165,6,21,1009.118438,1647.534396,6.576923,638.415958
166,6,22,1703.301242,632.444572,6.576923,-1070.856670


In [449]:
import plotly.graph_objects as go

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=hourON["Proportion_of_consumption_production"],
    x=hourON["HourOfTimestamp"],
    y=hourON["WeekdayOfTimestamp"], colorscale="RdBu", zmid=0
)
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    yaxis_nticks=10
    # annotations=annotations,
)

fig.show()

In [450]:
agg_consum_prod["Proportion_of_consumption_production"] = agg_consum_prod["Mean_production"] - agg_consum_prod[
    "Mean_consumption"]
hourON = agg_consum_prod.groupby(["MonthOfTimestamp", "WeekdayOfTimestamp"]).mean()
hourON.reset_index(inplace=True)

hourON

,MonthOfTimestamp,WeekdayOfTimestamp,Mean_consumption,Mean_production,HourOfTimestamp,Proportion_of_consumption_production
0,1,0,1752.714828,1348.283324,11.5,-404.431504
1,1,1,2883.953572,874.349477,11.5,-2009.604095
2,1,2,2880.152406,1427.859590,11.5,-1452.292815
3,1,3,2731.984599,1291.848087,11.5,-1440.136512
4,1,4,2588.583833,982.300648,11.5,-1606.283185
...,...,...,...,...,...,...
79,12,2,1871.091689,914.980412,11.5,-956.111277
80,12,3,1856.282038,924.989465,11.5,-931.292573
81,12,4,1489.786027,204.082466,11.5,-1285.703561
82,12,5,1740.727360,893.782153,11.5,-846.945207


In [451]:
import plotly.graph_objects as go

#Create heatmap
fig = go.Figure(data=go.Heatmap(
    z=hourON["Proportion_of_consumption_production"],
    x=hourON["WeekdayOfTimestamp"],
    y=hourON["MonthOfTimestamp"], colorscale="RdBu", zmid=0
)
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=10,
    yaxis_nticks=20
    # annotations=annotations,
)

fig.show()

In [452]:
line_data = hourON.groupby(["HourOfTimestamp"]).mean()
line_data.reset_index()

,HourOfTimestamp,MonthOfTimestamp,WeekdayOfTimestamp,Mean_consumption,Mean_production,Proportion_of_consumption_production
0,11.120000,7.000000,1.000000,1439.746587,1908.203729,468.457142
1,11.500000,6.487805,3.060976,1544.330297,1573.647015,29.316718
2,11.844828,7.000000,0.000000,1486.093064,2271.073920,784.980855


In [453]:
fig = go.Figure(data=[
    go.Line(name='Mean_consumption', y=(line_data["Mean_consumption"])),
    go.Line(name='Mean_production', y=line_data["Mean_production"]),
    go.Line(name='Mean_energy_balance', y=(line_data["Mean_production"] - line_data["Mean_consumption"]))
]
)

fig.update_layout(margin=dict(
    l=30,
    r=30,
    b=30,
    t=50,
), xaxis_nticks=30,
    yaxis_nticks=20
    # annotations=annotations,
)

fig.show()

/Users/christiannielsen/Documents/Conda_environments/lib/python3.9/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


